In [1]:

######## snakemake preamble start (automatically inserted, do not edit) ########
import sys; sys.path.extend(['/fh/fast/bloom_j/software/miniconda3/envs/barcoded_flu_pdmH1N1/lib/python3.8/site-packages', '/fh/fast/bloom_j/computational_notebooks/dbacsik/2019/flu_pdmH1N1_barcode_hashing/notebooks']); import pickle; snakemake = pickle.loads(b'\x80\x04\x95\x84\x1f\x00\x00\x00\x00\x00\x00\x8c\x10snakemake.script\x94\x8c\tSnakemake\x94\x93\x94)\x81\x94}\x94(\x8c\x05input\x94\x8c\x0csnakemake.io\x94\x8c\nInputFiles\x94\x93\x94)\x81\x94(\x8c_results/viral_tags_bcs_in_cells/scProgenyProduction_trial3_cell_barcodes_with_viral_tags.csv.gz\x94\x8cLresults/viral_fastq10x/scProgenyProduction_trial3_viral_genes_by_cell.csv.gz\x94\x8cOresults/viral_fastq10x/scProgenyProduction_trial3_viral_bc_by_cell_valid.csv.gz\x94\x8cQresults/viral_progeny/scProgenyProduction_trial3_filtered_progeny_viral_bc.csv.gz\x94\x8cTresults/viral_fastq10x/scProgenyProduction_trial3_contributes_progeny_by_cell.csv.gz\x94e}\x94(\x8c\x06_names\x94}\x94(\x8c\x10cell_annotations\x94K\x00N\x86\x94\x8c\x17viral_genes_by_cell_csv\x94K\x01N\x86\x94\x8c\x18viral_barcodes_valid_csv\x94K\x02N\x86\x94\x8c\x1dfiltered_progeny_viral_bc_csv\x94K\x03N\x86\x94\x8c\x1fcontributes_progeny_by_cell_csv\x94K\x04N\x86\x94u\x8c\x12_allowed_overrides\x94]\x94(\x8c\x05index\x94\x8c\x04sort\x94eh\x1e\x8c\tfunctools\x94\x8c\x07partial\x94\x93\x94h\x06\x8c\x19Namedlist._used_attribute\x94\x93\x94\x85\x94R\x94(h$)}\x94\x8c\x05_name\x94h\x1esNt\x94bh\x1fh"h$\x85\x94R\x94(h$)}\x94h(h\x1fsNt\x94bh\x12h\nh\x14h\x0bh\x16h\x0ch\x18h\rh\x1ah\x0eub\x8c\x06output\x94h\x06\x8c\x0bOutputFiles\x94\x93\x94)\x81\x94\x8cGresults/viral_fastq10x/scProgenyProduction_trial3_integrate_data.csv.gz\x94a}\x94(h\x10}\x94\x8c\x13integrated_data_csv\x94K\x00N\x86\x94sh\x1c]\x94(h\x1eh\x1feh\x1eh"h$\x85\x94R\x94(h$)}\x94h(h\x1esNt\x94bh\x1fh"h$\x85\x94R\x94(h$)}\x94h(h\x1fsNt\x94bh5h2ub\x8c\x06params\x94h\x06\x8c\x06Params\x94\x93\x94)\x81\x94]\x94(\x8c\x05fluHA\x94\x8c\x05fluNA\x94ea}\x94(h\x10}\x94\x8c\x14barcoded_viral_genes\x94K\x00N\x86\x94sh\x1c]\x94(h\x1eh\x1feh\x1eh"h$\x85\x94R\x94(h$)}\x94h(h\x1esNt\x94bh\x1fh"h$\x85\x94R\x94(h$)}\x94h(h\x1fsNt\x94bhIhDub\x8c\twildcards\x94h\x06\x8c\tWildcards\x94\x93\x94)\x81\x94\x8c\x1ascProgenyProduction_trial3\x94a}\x94(h\x10}\x94\x8c\x04expt\x94K\x00N\x86\x94sh\x1c]\x94(h\x1eh\x1feh\x1eh"h$\x85\x94R\x94(h$)}\x94h(h\x1esNt\x94bh\x1fh"h$\x85\x94R\x94(h$)}\x94h(h\x1fsNt\x94b\x8c\x04expt\x94hXub\x8c\x07threads\x94K\x01\x8c\tresources\x94h\x06\x8c\tResources\x94\x93\x94)\x81\x94(K\x01K\x01e}\x94(h\x10}\x94(\x8c\x06_cores\x94K\x00N\x86\x94\x8c\x06_nodes\x94K\x01N\x86\x94uh\x1c]\x94(h\x1eh\x1feh\x1eh"h$\x85\x94R\x94(h$)}\x94h(h\x1esNt\x94bh\x1fh"h$\x85\x94R\x94(h$)}\x94h(h\x1fsNt\x94bhnK\x01hpK\x01ub\x8c\x03log\x94h\x06\x8c\x03Log\x94\x93\x94)\x81\x94\x8c<results/logs/integrate_data_scProgenyProduction_trial3.ipynb\x94a}\x94(h\x10}\x94\x8c\x08notebook\x94K\x00N\x86\x94sh\x1c]\x94(h\x1eh\x1feh\x1eh"h$\x85\x94R\x94(h$)}\x94h(h\x1esNt\x94bh\x1fh"h$\x85\x94R\x94(h$)}\x94h(h\x1fsNt\x94bh\x82h\x7fub\x8c\x06config\x94}\x94(\x8c\x08max_cpus\x94K\x10\x8c\x07log_dir\x94\x8c\x0cresults/logs\x94\x8c\x0cfastq10x_dir\x94\x8c\x10results/fastq10x\x94\x8c\npacbio_dir\x94\x8c\x0eresults/pacbio\x94\x8c\x0emkfastq10x_dir\x94\x8c\x1fresults/fastq10x/mkfastq_output\x94\x8c\ngenome_dir\x94\x8c\x0fresults/genomes\x94\x8c\trefgenome\x94\x8c\x19results/genomes/refgenome\x94\x8c\x14aligned_fastq10x_dir\x94\x8c\x18results/aligned_fastq10x\x94\x8c\x12viral_fastq10x_dir\x94\x8c\x16results/viral_fastq10x\x94\x8c\x1bviral_tags_bcs_in_cells_dir\x94\x8c\x1fresults/viral_tags_bcs_in_cells\x94\x8c\x11viral_progeny_dir\x94\x8c\x15results/viral_progeny\x94\x8c\x13align_and_parse_dir\x94\x8c#results/pacbio/CA09_align_and_parse\x94\x8c\x0fcell_genome_ftp\x94\x8cmftp://ftp.ensembl.org/pub/release-98/fasta/canis_familiaris/dna/Canis_familiaris.CanFam3.1.dna.toplevel.fa.gz\x94\x8c\x0ccell_gtf_ftp\x94\x8c^ftp://ftp.ensembl.org/pub/release-98/gtf/canis_familiaris/Canis_familiaris.CanFam3.1.98.gtf.gz\x94\x8c\x0cviral_genome\x94\x8c!data/flu_sequences/flu-CA09.fasta\x94\x8c\tviral_gtf\x94\x8c\x1fdata/flu_sequences/flu-CA09.gtf\x94\x8c\rviral_genbank\x94\x8c\x1edata/flu_sequences/flu-CA09.gb\x94\x8c\x14viral_tag_identities\x94\x8c+data/flu_sequences/flu-CA09_viral_tags.yaml\x94\x8c\x0fviral_amplicons\x94\x8c.data/flu_sequences/pacbio_amplicons/fluCA09.gb\x94\x8c\x11amplicon_features\x94\x8c9data/flu_sequences/pacbio_amplicons/fluCA09_features.yaml\x94\x8c\x1eamplicon_linearization_primers\x94\x8c=data/flu_sequences/pacbio_amplicons/linearization_primers.tsv\x94\x8c\x15amplicon_to_reference\x94\x8c=data/flu_sequences/pacbio_amplicons/amplicon_to_reference.csv\x94\x8c\x0escoreGapNoncan\x94J\xfc\xff\xff\xff\x8c\x0cscoreGapGCAG\x94J\xfc\xff\xff\xff\x8c\x0cscoreGapATAC\x94J\xfc\xff\xff\xff\x8c\x14cb_whitelist_10x_url\x94\x8ckhttps://github.com/10XGenomics/cellranger/raw/master/lib/python/cellranger/barcodes/3M-february-2018.txt.gz\x94\x8c\x10cb_whitelist_10x\x94\x8c-results/aligned_fastq10x/cb_whitelist_10x.txt\x94\x8c\ncb_len_10x\x94K\x10\x8c\x0bumi_len_10x\x94K\x0c\x8c\x14total_UMI_deviations\x94G@\x04\x00\x00\x00\x00\x00\x00\x8c\x13infection_threshold\x94G?\x84z\xe1G\xae\x14{\x8c\rviral_tag_fdr\x94G?\x1a6\xe2\xeb\x1cC-\x8c\x1dviral_barcode_upstream_length\x94K\x1c\x8c\x16viral_barcode_mismatch\x94K\x00\x8c\x12viral_barcode_minq\x94K\x1e\x8c\x17progeny_detection_limit\x94\x8c\x041e-5\x94\x8c\x0cviral_bc_fdr\x94G?\x1a6\xe2\xeb\x1cC-\x8c\x0eccs_min_length\x94K2\x8c\x0eccs_max_length\x94M\x88\x13\x8c\nccs_min_rq\x94G?\xef\xf7\xce\xd9\x16\x87+\x8c\x0bexperiments\x94}\x94\x8c\x1ascProgenyProduction_trial3\x94}\x94(\x8c\x0bdescription\x94X\x1b\x01\x00\x00Single-cell transcriptomics performed on even lower MOI-infected sample. Innoculum volumes based on `scProgenyProduction_trial1` and `scProgenyProduction_trial2` data. Loaded all collected cells from two 24-well wells, each infected with either wildtype or dblySn viral tag variants.\x94\x8c\tlab_notes\x94\x8c0https://benchling.com/s/etr-N4laiIAy3AUePJdig7wR\x94\x8c\rexpect_ncells\x94M\x10\'\x8c\x0ftranscriptomics\x94}\x94(\x8c\x08datetime\x94\x8c\x04date\x94\x93\x94C\x04\x07\xe5\x03\x0c\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-TT-A3\x94\x8c\nbcl_folder\x94\x8c_/shared/ngs/illumina/bloom_lab/210312_D00300_1192_BHKHNYBCX3/raw/210312_D00300_1192_BHKHNYBCX3/\x94\x8c\x04lane\x94K\x01\x8c\x10index_sequencing\x94\x8c\x06single\x94uh\xdcC\x04\x07\xe5\x04\t\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-TT-A3\x94\x8c\nbcl_folder\x94\x8c_/shared/ngs/illumina/bloom_lab/210409_D00300_1210_BHKC7KBCX3/raw/210409_D00300_1210_BHKC7KBCX3/\x94\x8c\x04lane\x94\x8c\x01*\x94\x8c\x10index_sequencing\x94\x8c\x06single\x94uh\xdcC\x04\x07\xe5\x08\x19\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-TT-A3\x94\x8c\nbcl_folder\x94\x8c_/shared/ngs/illumina/bloom_lab/210825_D00300_1310_BHMH5TBCX3/raw/210825_D00300_1310_BHMH5TBCX3/\x94\x8c\x04lane\x94h\xf1\x8c\x10index_sequencing\x94\x8c\x04none\x94uh\xdcC\x04\x07\xe5\t\x0f\x94\x85\x94R\x94}\x94(\x8c\x05index\x94\x8c\x08SI-TT-A3\x94\x8c\nbcl_folder\x94\x8cY/shared/ngs/illumina/bloom_lab/210915_VH00699_2_AAAMCL7M5/raw/210915_VH00699_2_AAAMCL7M5/\x94\x8c\x04lane\x94h\xf1\x8c\x10index_sequencing\x94\x8c\x04none\x94uu\x8c\x17pacbio_viral_sequencing\x94}\x94\x8c\x1c2021-08-05_all_segments_run1\x94\x8cp/fh/fast/bloom_j/SR/ngs/pacbio/210730_SequelIIe/r64272e_20210730_182729/1_A01/m64272e_210730_193026.subreads.bam\x94s\x8c\x0eviral_barcodes\x94}\x94(\x8c\x0bsupernatant\x94}\x94(\x8c\x02wt\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xdcC\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x90/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/WT-sup-fluHA-rep1_S9_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xdcC\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x91/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/WT-sup-fluHA-rep2_S13_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xdcC\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x90/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/WT-sup-fluNA-rep1_S1_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xdcC\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x86/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/WT-sup-fluNA-rep2_S1_L001_R1_001.fastq.gz\x94suu\x8c\x03syn\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xdcC\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x95/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/dblSyn-sup-fluHA-rep1_S10_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xdcC\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x95/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/dblSyn-sup-fluHA-rep2_S14_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xdcC\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x94/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/dblSyn-sup-fluNA-rep1_S2_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xdcC\x04\x07\xe5\x04\x08\x94\x85\x94R\x94\x8c\x94/shared/ngs/illumina/bloom_lab/210408_M04866_0464_000000000-JL5M4_new-demux/Data/Intensities/BaseCalls/dblSyn-sup-fluNA-rep2_S6_L001_R1_001.fastq.gz\x94suuu\x8c\x10second_infection\x94}\x94(\x8c\x02wt\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xdcC\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x85/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/WT-SI-fluHA-rep1_S2_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xdcC\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x85/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/WT-SI-fluHA-rep2_S3_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xdcC\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x86/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/WT-SI-fluNA-rep1_S11_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xdcC\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x86/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/WT-SI-fluNA-rep2_S12_L001_R1_001.fastq.gz\x94suu\x8c\x03syn\x94}\x94(\x8c\x05fluHA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xdcC\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x89/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/dblSyn-SI-fluHA-rep1_S4_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xdcC\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x89/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/dblSyn-SI-fluHA-rep2_S5_L001_R1_001.fastq.gz\x94su\x8c\x05fluNA\x94}\x94(\x8c\x0breplicate_1\x94}\x94h\xdcC\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x8a/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/dblSyn-SI-fluNA-rep1_S13_L001_R1_001.fastq.gz\x94s\x8c\x0breplicate_2\x94}\x94h\xdcC\x04\x07\xe5\x05\x11\x94\x85\x94R\x94\x8c\x8a/shared/ngs/illumina/bloom_lab/210517_M03100_0681_000000000-JL4TM/Data/Intensities/BaseCalls/dblSyn-SI-fluNA-rep2_S14_L001_R1_001.fastq.gz\x94suuuuusu\x8c\x04rule\x94\x8c\x0eintegrate_data\x94\x8c\x0fbench_iteration\x94N\x8c\tscriptdir\x94\x8c[/fh/fast/bloom_j/computational_notebooks/dbacsik/2019/flu_pdmH1N1_barcode_hashing/notebooks\x94ub.'); from snakemake.logging import logger; logger.printshellcmds = False; import os; os.chdir('/fh/fast/bloom_j/computational_notebooks/dbacsik/2019/flu_pdmH1N1_barcode_hashing');
######## snakemake preamble end #########


# Integrate data
This notebook takes processed data and integrates it into a single dataframe.

The general structure of the dataframe is that each cell barcode is listed on a row, and features of that cell are listed in columns. Cells that have more than one valid viral barcode identified may have multiple rows--one for each valid viral barcode.

Import python modules:


In [2]:
from IPython.display import display

import pandas as pd

Variables:

In [3]:
# Input sources
cell_annotations_csv = snakemake.input.cell_annotations
viral_genes_by_cell_csv = snakemake.input.viral_genes_by_cell_csv
pacbio_consensus_gene_csv = 'results/pacbio/scProgenyProduction_trial3_consensus_gene.csv.gz' # snakemake.input.pacbio_consensus_gene_csv
viral_barcodes_valid_csv = snakemake.input.viral_barcodes_valid_csv
filtered_progeny_viral_bc_csv = snakemake.input.filtered_progeny_viral_bc_csv
contributes_progeny_by_cell_csv = snakemake.input.contributes_progeny_by_cell_csv

# # Params and wildcards
barcoded_viral_genes = snakemake.params.barcoded_viral_genes
expt = snakemake.wildcards.expt

# Output
integrated_data_csv = snakemake.output.integrated_data_csv

## Load data and transform into wide format
Load data from the following sources:  
* `cell_annotations_csv` contains a list of all cell barcodes, their infection status/tag, total_UMIs, viral_UMIs, and frac_viral_UMIs  

* `viral_genes_by_cell_csv` contains every cell barcode, each viral gene, whether the viral gene is present, the fraction of UMIs from that gene, and the total number of viral genes detected in that cell  

* `viral_barcodes_valid_csv` contains each infected cell, the valid viral barcodes detected in that cell, viral_bc_UMIs, and frac_viral_bc_UMIs  

* `filtered_progeny_viral_bc_csv` contains the progeny source, the tag of the progeny source, the valid viral barcodes detected in that progeny source, and the frequency of the viral barcodes in that progeny source  

* `pacbio_consensus_gene_csv` contains the cell barcode, each viral gene, and the consensus genotype of the viral gene in that cell

Some of these dataframes are in a long format, with many rows per cell barcode. I will transform them into a wide format, so that each cell barcode has only one row. The only exception is the viral barcodes, which will have one row per cell barcode-viral barcode pair.

Load cell barcodes, tags, and basic metrics for every cell in transcriptome:

In [4]:
cell_annotations = pd.read_csv(cell_annotations_csv)
cell_annotations = (
    cell_annotations
    [['cell_barcode',
      'infected',
      'infecting_viral_tag',
      'total_UMIs',
      'viral_UMIs',
      'frac_viral_UMIs']])

display(cell_annotations)

,cell_barcode,infected,infecting_viral_tag,total_UMIs,viral_UMIs,frac_viral_UMIs
0,AAACCCAGTAACAAGT,uninfected,none,47873,6,0.000125
1,AAACCCATCATTGCTT,uninfected,none,90114,10,0.000111
2,AAACGAAAGATGTTGA,uninfected,none,111630,18,0.000161
3,AAACGAAGTACTTCCC,infected,both,56828,24082,0.423770
4,AAACGAAGTAGACGTG,infected,wt,124341,4654,0.037429
...,...,...,...,...,...,...
3124,TTTGATCTCAGTCATG,uninfected,none,44890,4,0.000089
3125,TTTGATCTCCCGTTCA,uninfected,none,63150,3,0.000048
3126,TTTGGAGAGTTGCCTA,uninfected,none,65941,12,0.000182
3127,TTTGGAGGTATCGTTG,infected,wt,150130,3526,0.023486


Load viral genes detected in each cell:

In [5]:
viral_genes = pd.read_csv(viral_genes_by_cell_csv)
viral_genes = viral_genes.rename(columns={'frac_gene_UMIs': 'frac_UMIs',
                                          'gene_present': 'present'})
viral_genes = (viral_genes
               .pivot(
    index=['cell_barcode', 'n_viral_genes'],
    columns=['gene'],
    values=['frac_UMIs', 'present']))
viral_genes.columns = ['_'.join(col).strip() for col in viral_genes.columns.values]
viral_genes = viral_genes.reset_index()
display(viral_genes)

,cell_barcode,n_viral_genes,frac_UMIs_fluHA,frac_UMIs_fluM,frac_UMIs_fluNA,frac_UMIs_fluNP,frac_UMIs_fluNS,frac_UMIs_fluPA,frac_UMIs_fluPB1,frac_UMIs_fluPB2,present_fluHA,present_fluM,present_fluNA,present_fluNP,present_fluNS,present_fluPA,present_fluPB1,present_fluPB2
0,AAACCCAGTAACAAGT,0,4.17772e-05,6.26658e-05,0,0,2.08886e-05,0,0,0,False,False,False,False,False,False,False,False
1,AAACCCATCATTGCTT,1,0,5.54853e-05,0,3.32912e-05,1.10971e-05,0,0,1.10971e-05,False,False,False,False,False,False,False,True
2,AAACGAAAGATGTTGA,0,2.68745e-05,8.95817e-05,0,8.95817e-06,3.58327e-05,0,0,0,False,False,False,False,False,False,False,False
3,AAACGAAGTACTTCCC,7,3.51939e-05,0.229816,0.00739072,0.0728338,0.10157,0.00040473,0.00890406,0.00281551,False,True,True,True,True,True,True,True
4,AAACGAAGTAGACGTG,8,0.00256553,0.0127231,0.000249314,0.00488978,0.0165271,8.84664e-05,0.000361908,2.41272e-05,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3124,TTTGATCTCAGTCATG,0,0,8.91067e-05,0,0,0,0,0,0,False,False,False,False,False,False,False,False
3125,TTTGATCTCCCGTTCA,0,0,1.58353e-05,0,1.58353e-05,1.58353e-05,0,0,0,False,False,False,False,False,False,False,False
3126,TTTGGAGAGTTGCCTA,1,0,9.09904e-05,1.51651e-05,1.51651e-05,6.06603e-05,0,0,0,False,False,True,False,False,False,False,False
3127,TTTGGAGGTATCGTTG,8,0.000619463,0.0111104,0.00029974,0.00278425,0.0081929,4.66263e-05,0.000379671,5.32872e-05,True,True,True,True,True,True,True,True


Load viral gene genotypes:

In [6]:
viral_genotypes = pd.read_csv(pacbio_consensus_gene_csv)
viral_genotypes = viral_genotypes.rename(columns={'total_UMI': 'pacbio_UMIs',
                                                  'consensus_mutations': 'mutations'})
viral_genotypes = (
    viral_genotypes
    .pivot(index=['cell_barcode'],
           columns=['gene'],
           values=['mutations', 'pacbio_UMIs']))
viral_genotypes.columns = ['_'.join(col).strip() for col in viral_genotypes.columns.values]
viral_genotypes = viral_genotypes.reset_index()
display(viral_genotypes)

,cell_barcode,mutations_fluHA,mutations_fluM,mutations_fluNA,mutations_fluNP,mutations_fluNS,mutations_fluPA,mutations_fluPB1,mutations_fluPB2,pacbio_UMIs_fluHA,pacbio_UMIs_fluM,pacbio_UMIs_fluNA,pacbio_UMIs_fluNP,pacbio_UMIs_fluNS,pacbio_UMIs_fluPA,pacbio_UMIs_fluPB1,pacbio_UMIs_fluPB2
0,AAACGAAGTACTTCCC,NaN,WT,WT,WT,WT,WT,WT,WT,NaN,435,161,828,467,10,346,54
1,AAACGAAGTAGACGTG,WT,WT,WT,WT,WT,WT,WT,NaN,36,49,16,247,202,5,41,NaN
2,AAAGGATTCTGATGGT,WT,WT,WT,WT,WT,WT,WT,del557to2280,13,36,327,70,97,4,37,1
3,AAAGGGCTCCGCACTT,WT,WT,WT,WT,WT,NaN,WT,del168to2007 del2097to2106,9,21,15,80,50,NaN,13,1062
4,AAAGTCCAGTAGAGTT,WT,WT,WT,WT,WT,WT,WT,WT,27,308,27,1088,253,37,760,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,TTGGGATGTAACGATA,WT,WT,WT,WT,WT,WT,WT,WT,32,181,29,295,360,6,55,6
406,TTGGGATTCCGATTAG,WT,WT,WT,WT,WT,G885T,WT,WT,30,131,23,259,149,1,94,6
407,TTTACCAGTCGCTTAA,WT,WT,WT,WT,WT,NaN,WT,NaN,7,12,4,39,46,NaN,6,NaN
408,TTTCACAAGCCAAGCA,T465A,WT,WT,WT,NaN,WT,NaN,WT,78,32,113,158,NaN,88,NaN,156


Load valid viral barcodes in each infected cell:

In [7]:
transcriptome_viral_barcodes = pd.read_csv(viral_barcodes_valid_csv)
transcriptome_viral_barcodes = transcriptome_viral_barcodes.drop(columns=['valid_viral_bc'])
transcriptome_viral_barcodes = transcriptome_viral_barcodes.rename(columns={'gene': 'barcoded_gene'})
assert set(transcriptome_viral_barcodes['barcoded_gene']) == set(barcoded_viral_genes), \
       "Barcoded genes in barcode counts do not match expectation."
display(transcriptome_viral_barcodes)

,cell_barcode,barcoded_gene,viral_barcode,viral_bc_UMIs,frac_viral_bc_UMIs
0,AAACGAAGTAGACGTG,fluHA,AAGTAAGCGACATGAG,251,0.002019
1,AAAGGATTCTGATGGT,fluHA,GTGGAGTCGCCAGTTC,114,0.001424
2,AAAGGGCTCCGCACTT,fluHA,AATCTACGAGGGAAAC,33,0.000257
3,AAAGGGCTCCGCACTT,fluHA,ATGGATCAGATTTCCT,10,0.000078
4,AAAGGGCTCCGCACTT,fluHA,GCAAAAATACAAAGTA,28,0.000218
...,...,...,...,...,...
895,TTGGGATTCCGATTAG,fluNA,GTTTCGTGAGCATGTA,25,0.000182
896,TTTACCAGTCGCTTAA,fluNA,ATTTGGATGTAGGGGC,4,0.000035
897,TTTACCAGTCGCTTAA,fluNA,TTGGAGGAGACCCGTG,7,0.000061
898,TTTCACAAGCCAAGCA,fluNA,GGTATCAGTTATTGTT,186,0.002718


Load progeny viral barcode frequencies:

In [8]:
progeny_viral_barcodes = pd.read_csv(filtered_progeny_viral_bc_csv)
progeny_viral_barcodes = progeny_viral_barcodes.drop(columns=['Unnamed: 0'])
progeny_viral_barcodes = (progeny_viral_barcodes
                          .rename(columns={'tag': 'infecting_viral_tag',
                                           'average_freq': 'freq',
                                           'gene': 'barcoded_gene'}))
assert set(progeny_viral_barcodes['barcoded_gene']) == set(barcoded_viral_genes), \
       "Barcoded genes in barcode counts do not match expectation."
progeny_viral_barcodes = (
    progeny_viral_barcodes
    .pivot(index=['viral_barcode', 'infecting_viral_tag', 'barcoded_gene'],
           columns=['source'],
           values=['freq']))
progeny_viral_barcodes.columns = ['_'.join(col).strip() for col in progeny_viral_barcodes.columns.values]
progeny_viral_barcodes = progeny_viral_barcodes.reset_index()
display(progeny_viral_barcodes)

,viral_barcode,infecting_viral_tag,barcoded_gene,freq_second_infection,freq_supernatant
0,AAAAACACGTTCTATA,wt,fluNA,0.000010,0.000010
1,AAAACATGATGACGCC,wt,fluNA,0.000010,0.000010
2,AAAACTAGTTAGAGCA,wt,fluHA,0.000010,0.000010
3,AAAAGTTCTTGGATGT,wt,fluNA,0.000010,0.000010
4,AAAATTGTCCAATATA,syn,fluHA,0.000010,0.000010
...,...,...,...,...,...
790,TTTCGTGAATCACTGC,wt,fluHA,0.000010,0.000073
791,TTTCTTTACTCAGAAT,wt,fluNA,0.003833,0.002060
792,TTTGCACACGTCTAAT,wt,fluHA,0.000010,0.000010
793,TTTTAACGTTATACTA,wt,fluNA,0.000010,0.000010


Load whether each infected cell contributes any progeny:

In [9]:
contributes_progeny = pd.read_csv(contributes_progeny_by_cell_csv)
contributes_progeny = contributes_progeny.rename(columns={'gene': 'barcoded_gene',
                                                          'max_progeny_freq': 'max_freq'})
assert set(transcriptome_viral_barcodes['barcoded_gene']) == set(barcoded_viral_genes), \
       "Barcoded genes in barcode counts do not match expectation."
contributes_progeny = (
    contributes_progeny
    .pivot(index=['cell_barcode'],
           columns=['source'],
           values=['max_freq', 'contributes_progeny']))
contributes_progeny.columns = ['_'.join(col).strip() for col in contributes_progeny.columns.values]
contributes_progeny = contributes_progeny.reset_index()
display(contributes_progeny)

,cell_barcode,max_freq_second_infection,max_freq_supernatant,contributes_progeny_second_infection,contributes_progeny_supernatant
0,AAACGAAGTAGACGTG,1e-05,7.62399e-05,False,True
1,AAAGGATTCTGATGGT,0.077933,0.0131834,True,True
2,AAAGGGCTCCGCACTT,0.000124415,0.00195048,True,True
3,AAAGTCCAGTAGAGTT,1e-05,1e-05,False,False
4,AACCAACAGGGTCTTT,2.17873e-05,0.000971284,True,True
...,...,...,...,...,...
354,TTGGGATGTAACGATA,0.000153675,0.00128394,True,True
355,TTGGGATTCCGATTAG,1e-05,1e-05,False,False
356,TTTACCAGTCGCTTAA,1.8581e-05,0.00121169,True,True
357,TTTCACAAGCCAAGCA,1e-05,1e-05,False,False


## Integrate data into single dataframe

Start with `cell_annotations` dataframe. Bring in viral gene expression information.

In [10]:
integrated_df = pd.merge(
    left=cell_annotations,
    right=viral_genes,
    on='cell_barcode',
    how='outer',
    validate='one_to_one'
)
display(integrated_df)

,cell_barcode,infected,infecting_viral_tag,total_UMIs,viral_UMIs,frac_viral_UMIs,n_viral_genes,frac_UMIs_fluHA,frac_UMIs_fluM,frac_UMIs_fluNA,...,frac_UMIs_fluPB1,frac_UMIs_fluPB2,present_fluHA,present_fluM,present_fluNA,present_fluNP,present_fluNS,present_fluPA,present_fluPB1,present_fluPB2
0,AAACCCAGTAACAAGT,uninfected,none,47873,6,0.000125,0,4.17772e-05,6.26658e-05,0,...,0,0,False,False,False,False,False,False,False,False
1,AAACCCATCATTGCTT,uninfected,none,90114,10,0.000111,1,0,5.54853e-05,0,...,0,1.10971e-05,False,False,False,False,False,False,False,True
2,AAACGAAAGATGTTGA,uninfected,none,111630,18,0.000161,0,2.68745e-05,8.95817e-05,0,...,0,0,False,False,False,False,False,False,False,False
3,AAACGAAGTACTTCCC,infected,both,56828,24082,0.423770,7,3.51939e-05,0.229816,0.00739072,...,0.00890406,0.00281551,False,True,True,True,True,True,True,True
4,AAACGAAGTAGACGTG,infected,wt,124341,4654,0.037429,8,0.00256553,0.0127231,0.000249314,...,0.000361908,2.41272e-05,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3124,TTTGATCTCAGTCATG,uninfected,none,44890,4,0.000089,0,0,8.91067e-05,0,...,0,0,False,False,False,False,False,False,False,False
3125,TTTGATCTCCCGTTCA,uninfected,none,63150,3,0.000048,0,0,1.58353e-05,0,...,0,0,False,False,False,False,False,False,False,False
3126,TTTGGAGAGTTGCCTA,uninfected,none,65941,12,0.000182,1,0,9.09904e-05,1.51651e-05,...,0,0,False,False,True,False,False,False,False,False
3127,TTTGGAGGTATCGTTG,infected,wt,150130,3526,0.023486,8,0.000619463,0.0111104,0.00029974,...,0.000379671,5.32872e-05,True,True,True,True,True,True,True,True


Bring in viral gene genotypes. Many cells do not have pacbio data. For all missing data, fill all counts with 0 and fill all mutations with "Not Detected".

In [11]:
integrated_df = pd.merge(
    left=integrated_df,
    right=viral_genotypes,
    on='cell_barcode',
    how='left',
    validate='one_to_one'
)

# Fill empty mutations columns with "Not Detected"
integrated_df[[col for col in integrated_df if col.startswith('mutations')]] = (
     integrated_df[[col for col in integrated_df if col.startswith('mutations')]]
     .fillna('Not Detected'))
# Fill empty pacbio_UMIs columns with 0
integrated_df[[col for col in integrated_df if col.startswith('pacbio_UMIs')]] = (
     integrated_df[[col for col in integrated_df if col.startswith('pacbio_UMIs')]]
     .fillna(0))

display(integrated_df)

,cell_barcode,infected,infecting_viral_tag,total_UMIs,viral_UMIs,frac_viral_UMIs,n_viral_genes,frac_UMIs_fluHA,frac_UMIs_fluM,frac_UMIs_fluNA,...,mutations_fluPB1,mutations_fluPB2,pacbio_UMIs_fluHA,pacbio_UMIs_fluM,pacbio_UMIs_fluNA,pacbio_UMIs_fluNP,pacbio_UMIs_fluNS,pacbio_UMIs_fluPA,pacbio_UMIs_fluPB1,pacbio_UMIs_fluPB2
0,AAACCCAGTAACAAGT,uninfected,none,47873,6,0.000125,0,4.17772e-05,6.26658e-05,0,...,Not Detected,Not Detected,0,0,0,0,0,0,0,0
1,AAACCCATCATTGCTT,uninfected,none,90114,10,0.000111,1,0,5.54853e-05,0,...,Not Detected,Not Detected,0,0,0,0,0,0,0,0
2,AAACGAAAGATGTTGA,uninfected,none,111630,18,0.000161,0,2.68745e-05,8.95817e-05,0,...,Not Detected,Not Detected,0,0,0,0,0,0,0,0
3,AAACGAAGTACTTCCC,infected,both,56828,24082,0.423770,7,3.51939e-05,0.229816,0.00739072,...,WT,WT,0,435,161,828,467,10,346,54
4,AAACGAAGTAGACGTG,infected,wt,124341,4654,0.037429,8,0.00256553,0.0127231,0.000249314,...,WT,Not Detected,36,49,16,247,202,5,41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3124,TTTGATCTCAGTCATG,uninfected,none,44890,4,0.000089,0,0,8.91067e-05,0,...,Not Detected,Not Detected,0,0,0,0,0,0,0,0
3125,TTTGATCTCCCGTTCA,uninfected,none,63150,3,0.000048,0,0,1.58353e-05,0,...,Not Detected,Not Detected,0,0,0,0,0,0,0,0
3126,TTTGGAGAGTTGCCTA,uninfected,none,65941,12,0.000182,1,0,9.09904e-05,1.51651e-05,...,Not Detected,Not Detected,0,0,0,0,0,0,0,0
3127,TTTGGAGGTATCGTTG,infected,wt,150130,3526,0.023486,8,0.000619463,0.0111104,0.00029974,...,WT,WT,13,76,33,249,116,7,47,8


Bring in valid viral barcodes for cells that have them:

In [12]:
integrated_df = pd.merge(
    left=integrated_df,
    right=transcriptome_viral_barcodes,
    on='cell_barcode',
    how='left',
    validate='one_to_many'
)
display(integrated_df)

,cell_barcode,infected,infecting_viral_tag,total_UMIs,viral_UMIs,frac_viral_UMIs,n_viral_genes,frac_UMIs_fluHA,frac_UMIs_fluM,frac_UMIs_fluNA,...,pacbio_UMIs_fluNA,pacbio_UMIs_fluNP,pacbio_UMIs_fluNS,pacbio_UMIs_fluPA,pacbio_UMIs_fluPB1,pacbio_UMIs_fluPB2,barcoded_gene,viral_barcode,viral_bc_UMIs,frac_viral_bc_UMIs
0,AAACCCAGTAACAAGT,uninfected,none,47873,6,0.000125,0,4.17772e-05,6.26658e-05,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
1,AAACCCATCATTGCTT,uninfected,none,90114,10,0.000111,1,0,5.54853e-05,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
2,AAACGAAAGATGTTGA,uninfected,none,111630,18,0.000161,0,2.68745e-05,8.95817e-05,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
3,AAACGAAGTACTTCCC,infected,both,56828,24082,0.423770,7,3.51939e-05,0.229816,0.00739072,...,161,828,467,10,346,54,NaN,NaN,NaN,NaN
4,AAACGAAGTAGACGTG,infected,wt,124341,4654,0.037429,8,0.00256553,0.0127231,0.000249314,...,16,247,202,5,41,0,fluHA,AAGTAAGCGACATGAG,251.0,0.002019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3677,TTTGATCTCCCGTTCA,uninfected,none,63150,3,0.000048,0,0,1.58353e-05,0,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
3678,TTTGGAGAGTTGCCTA,uninfected,none,65941,12,0.000182,1,0,9.09904e-05,1.51651e-05,...,0,0,0,0,0,0,NaN,NaN,NaN,NaN
3679,TTTGGAGGTATCGTTG,infected,wt,150130,3526,0.023486,8,0.000619463,0.0111104,0.00029974,...,33,249,116,7,47,8,fluHA,AGTAAACTTCCTCGCG,65.0,0.000433
3680,TTTGGAGGTATCGTTG,infected,wt,150130,3526,0.023486,8,0.000619463,0.0111104,0.00029974,...,33,249,116,7,47,8,fluNA,ACATCTTATTTACACG,39.0,0.000260


Bring in progeny frequencies:

In [13]:
integrated_df = pd.merge(
    left=integrated_df,
    right=progeny_viral_barcodes,
    on=['infecting_viral_tag', 'barcoded_gene', 'viral_barcode'],
    how='left',
    validate='many_to_one'
)
display(integrated_df)

,cell_barcode,infected,infecting_viral_tag,total_UMIs,viral_UMIs,frac_viral_UMIs,n_viral_genes,frac_UMIs_fluHA,frac_UMIs_fluM,frac_UMIs_fluNA,...,pacbio_UMIs_fluNS,pacbio_UMIs_fluPA,pacbio_UMIs_fluPB1,pacbio_UMIs_fluPB2,barcoded_gene,viral_barcode,viral_bc_UMIs,frac_viral_bc_UMIs,freq_second_infection,freq_supernatant
0,AAACCCAGTAACAAGT,uninfected,none,47873,6,0.000125,0,4.17772e-05,6.26658e-05,0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,AAACCCATCATTGCTT,uninfected,none,90114,10,0.000111,1,0,5.54853e-05,0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,AAACGAAAGATGTTGA,uninfected,none,111630,18,0.000161,0,2.68745e-05,8.95817e-05,0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,AAACGAAGTACTTCCC,infected,both,56828,24082,0.423770,7,3.51939e-05,0.229816,0.00739072,...,467,10,346,54,NaN,NaN,NaN,NaN,NaN,NaN
4,AAACGAAGTAGACGTG,infected,wt,124341,4654,0.037429,8,0.00256553,0.0127231,0.000249314,...,202,5,41,0,fluHA,AAGTAAGCGACATGAG,251.0,0.002019,0.000010,0.000076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3677,TTTGATCTCCCGTTCA,uninfected,none,63150,3,0.000048,0,0,1.58353e-05,0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3678,TTTGGAGAGTTGCCTA,uninfected,none,65941,12,0.000182,1,0,9.09904e-05,1.51651e-05,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3679,TTTGGAGGTATCGTTG,infected,wt,150130,3526,0.023486,8,0.000619463,0.0111104,0.00029974,...,116,7,47,8,fluHA,AGTAAACTTCCTCGCG,65.0,0.000433,0.000010,0.000695
3680,TTTGGAGGTATCGTTG,infected,wt,150130,3526,0.023486,8,0.000619463,0.0111104,0.00029974,...,116,7,47,8,fluNA,ACATCTTATTTACACG,39.0,0.000260,0.000028,0.001790


Bring in annotation of whether each infected cell contributes any progeny:

In [14]:
integrated_df = pd.merge(
    left=integrated_df,
    right=contributes_progeny,
    on=['cell_barcode'],
    how='left',
    validate='many_to_one'
)
display(integrated_df)

,cell_barcode,infected,infecting_viral_tag,total_UMIs,viral_UMIs,frac_viral_UMIs,n_viral_genes,frac_UMIs_fluHA,frac_UMIs_fluM,frac_UMIs_fluNA,...,barcoded_gene,viral_barcode,viral_bc_UMIs,frac_viral_bc_UMIs,freq_second_infection,freq_supernatant,max_freq_second_infection,max_freq_supernatant,contributes_progeny_second_infection,contributes_progeny_supernatant
0,AAACCCAGTAACAAGT,uninfected,none,47873,6,0.000125,0,4.17772e-05,6.26658e-05,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAACCCATCATTGCTT,uninfected,none,90114,10,0.000111,1,0,5.54853e-05,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAACGAAAGATGTTGA,uninfected,none,111630,18,0.000161,0,2.68745e-05,8.95817e-05,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAACGAAGTACTTCCC,infected,both,56828,24082,0.423770,7,3.51939e-05,0.229816,0.00739072,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAACGAAGTAGACGTG,infected,wt,124341,4654,0.037429,8,0.00256553,0.0127231,0.000249314,...,fluHA,AAGTAAGCGACATGAG,251.0,0.002019,0.000010,0.000076,1e-05,7.62399e-05,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3677,TTTGATCTCCCGTTCA,uninfected,none,63150,3,0.000048,0,0,1.58353e-05,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3678,TTTGGAGAGTTGCCTA,uninfected,none,65941,12,0.000182,1,0,9.09904e-05,1.51651e-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3679,TTTGGAGGTATCGTTG,infected,wt,150130,3526,0.023486,8,0.000619463,0.0111104,0.00029974,...,fluHA,AGTAAACTTCCTCGCG,65.0,0.000433,0.000010,0.000695,2.81273e-05,0.00179022,True,True
3680,TTTGGAGGTATCGTTG,infected,wt,150130,3526,0.023486,8,0.000619463,0.0111104,0.00029974,...,fluNA,ACATCTTATTTACACG,39.0,0.000260,0.000028,0.001790,2.81273e-05,0.00179022,True,True


## Check integrated dataframe

Check that the number of total cells has not changed.

In [15]:
assert integrated_df['cell_barcode'].nunique() == \
    len(cell_annotations['cell_barcode']), \
    "Total number of cells changed"

Check that every viral barcode has a `barcoded_gene`, `frac_viral_bc_UMIS`, `freq_supernatant`, and `freq_second_infection`.

In [16]:
assert len(integrated_df.query('viral_barcode.notnull()', engine='python')) == \
    len(integrated_df.query('barcoded_gene.notnull()', engine='python')) == \
    len(integrated_df.query('frac_viral_bc_UMIs.notnull()', engine='python')) == \
    len(integrated_df.query('freq_supernatant.notnull()', engine='python')) == \
    len(integrated_df.query('freq_second_infection.notnull()', engine='python')), \
    "Mismatch in viral barcode data."

## Output integrated dataframe

In [17]:
# save CSV
integrated_df.to_csv(integrated_data_csv, index=False)